<a href="https://colab.research.google.com/github/mahimathakkar1124/COVID-19-Research-Paper-Text-Summarization/blob/main/Data_606_Abstractive_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tika
!pip install PyPDF2
!pip install pytextrank
!pip install spacy && python -m spacy download en_core_web_sm
!pip install PyMuPDF
!pip install fitz
!pip install transformers
! pip install sentencepiece

  Created wheel for tika: filename=tika-1.24-cp37-none-any.whl size=32885 sha256=71afedcbc843785f7626d91b3b6c3dfc62ea5e864a5135be975b83ee26799cfc
  Stored in directory: /root/.cache/pip/wheels/73/9c/f5/0b1b738442fc2a2862bef95b908b374f8e80215550fb2a8975
Successfully built tika
     |████████████████████████████████| 81kB 3.9MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-cp37-none-any.whl size=61085 sha256=1d3f86fb753605772a1e608651a151cc841933f75d4fb595d35b727416a8072e
  Stored in directory: /root/.cache/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a229315c1b7
Successfully built PyPDF2
     |████████████████████████████████| 12.8MB 289kB/s 
     |████████████████████████████████| 460kB 43.7MB/s 
     |████████████████████████████████| 9.1MB 41.8MB/s 
     |████████████████████████████████| 1.1MB 48.3MB/s 
     |████████████████████████████████| 122kB 50.7MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-cp37-none-any.whl size=107098 sha256=7444fada7

# New Section

In [ ]:
import pandas as pd 
from glob import glob
from tika import parser # pip install tika
import re
import string
import nltk
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import spacy
import pytextrank
import numpy as np
import os
import textwrap
import ipywidgets as widgets
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from gensim.summarization import keywords, mz_keywords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from google.colab import drive
drive.mount('/content/gdrive/')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
Mounted at /content/gdrive/


In [ ]:
fileContent={}
for file1 in glob(f"/content/gdrive/My Drive/Data 606/*.pdf"):
  print(file1)
  raw = parser.from_file(file1)
  title=''
  #fileContent.append(raw['content'])
  #print(raw['content'])
  if 'dc:title' in raw['metadata'].keys():
    #print(raw['metadata']['dc:title'])
    title=raw['metadata']['dc:title']
  else:
    #continue
    title=os.path.basename(file1)
  fileContent[os.path.basename(file1)]={'Title': title , 'Content': raw['content']}
  


2021-04-13 15:07:33,263 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /tmp/tika-server.jar.


/content/gdrive/My Drive/Data 606/1-s2.0-S2352827321000264-main.pdf


2021-04-13 15:07:34,294 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to /tmp/tika-server.jar.md5.
2021-04-13 15:07:34,639 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


/content/gdrive/My Drive/Data 606/1-s2.0-S2213671121000886-main.pdf
/content/gdrive/My Drive/Data 606/1-s2.0-S2213671121000904-main.pdf
/content/gdrive/My Drive/Data 606/1-s2.0-S1517758021000023-main.pdf
/content/gdrive/My Drive/Data 606/1-s2.0-S1876201821000575-main.pdf
/content/gdrive/My Drive/Data 606/1-s2.0-S0378111921000901-main.pdf
/content/gdrive/My Drive/Data 606/1-s2.0-S0223523421001434-main.pdf
/content/gdrive/My Drive/Data 606/1-s2.0-S1018364721000276-main.pdf
/content/gdrive/My Drive/Data 606/1-s2.0-S2667147621000273-main.pdf
/content/gdrive/My Drive/Data 606/1-s2.0-S0020653921000368-main.pdf
/content/gdrive/My Drive/Data 606/1-s2.0-S241464472100004X-main.pdf
/content/gdrive/My Drive/Data 606/1-s2.0-S2667147621000182-main.pdf
/content/gdrive/My Drive/Data 606/1-s2.0-S0196655321000614-main.pdf
/content/gdrive/My Drive/Data 606/1-s2.0-S0885392421001718-main.pdf
/content/gdrive/My Drive/Data 606/1-s2.0-S0306987721000529-main.pdf
/content/gdrive/My Drive/Data 606/1-s2.0-S016344

In [ ]:
# Reference:- https://stackoverflow.com/questions/54664021/how-to-use-a-variable-as-the-input-for-a-form-field-in-google-colaboratory
titleList=[fileContent[keys]['Title'] for keys in fileContent.keys() ]
Title_Option = widgets.Dropdown(options=titleList, value=titleList[0])
Title_Option

Dropdown(options=('Low uptake of COVID-19 prevention behaviours and high socioeconomic impact of lockdown meas…

In [ ]:
Title_Option.value

'Low uptake of COVID-19 prevention behaviours and high socioeconomic impact of lockdown measures in South Asia: evidence from a large-scale multi-country surveillance programme'

In [ ]:
def remove_URL(text):
    url_pattern = re.compile(r'https?://\S+')
    return url_pattern.sub(r'', text)

In [ ]:
punct =[]
punct += list(string.punctuation)
punct += '’'
punct.remove("'")
punct.remove(".")
def remove_punctuations(text):
  #text= re.sub(re.compile('<.*?>'),'',text)
  for punctuation in punct:
         text = text.replace(punctuation, ' ')
  return text

In [ ]:
stop_words= stopwords.words('english')
# Add all the required words to the stop-words corpus
stop_words.extend(['from', 'mr', 'said','subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done',
  'try', 'many', 'applause', 'u','000', 'one', 'year', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'l ','2',
  'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come', 'title','abstract','preprint','biorxiv','al','et.',
  'author','copyright','holder','https','license','et','al.','may','also','medrxiv','granted','reuse','rights','used','reserved','peer','holder','figure','fig','table','http','j','doi',
  'jo','ur','na','l','p','ro', 'Published', 'Elsevier', 'Ltd'])
# Remove stop-words
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_words])

In [ ]:
def removing_digits_except_19(text):
  l=[]
  tokenized_text=word_tokenize(text)
  for item in tokenized_text:
    #print(type(item))
    if item == '19' or item == '2019':
      l.append(item)
    elif not item.isdigit():
      l.append(item)
  return ' '.join(l)


In [ ]:
def removing_short_word(text):
  long_words=[]
  for i in word_tokenize(text):
      #print(i)
      if len(i)>=3 or i=='.':                  #removing short word
          long_words.append(i)   
  return(" ".join(long_words).strip())

In [ ]:
def clean_text(text):
    print('Converting to lower case...')
    text=text.lower()
    #print(text)
    print('Removing /n characters...')
    text=text.replace('\n', ' ')
    #print(text)
    print('Removing urls...')
    text=remove_URL(text)
    #print(text)
    print('Removing punctuations...')
    text= remove_punctuations(text)
    #print(text)
    print("Removing digits....")
    text=removing_digits_except_19(text)
    print("Removing non Alpha-Numeric Characters")
    text=re.sub('[^A-Za-z0-9.]+',' ',text)
    #print(text)
    print('Removing the stopwords...')
    text=remove_stopwords(text)
    print('Removing Short words...')
    text=removing_short_word(text)
    #print(text)

    #print(text)
    return text


In [ ]:
#Reference:- https://stackoverflow.com/questions/15210148/get-parents-keys-from-nested-dictionary
def find_key(d, value):
    for k,v in d.items():
        if isinstance(v, dict):
            p = find_key(v, value)
            if p:
                return [k] + p
        elif v == value:
            return [k]

key_to_text=find_key(fileContent,Title_Option.value)[0]
original_text= fileContent[key_to_text]['Content']

cleaned_text=clean_text(original_text)

Converting to lower case...
Removing /n characters...
Removing urls...
Removing punctuations...
Removing digits....
Removing non Alpha-Numeric Characters
Removing the stopwords...
Removing Short words...


In [ ]:
cleaned_text

'low uptake covid prevention behaviours high socioeconomic impact lockdown measures south asia evidence large scale multi country surveillance programme journal pre proof low uptake covid prevention behaviours high socioeconomic impact lockdown measures south asia evidence large scale multi country surveillance programme dian kusuma rajendra pradeepa khadija khawaja mehedi hasan samreen siddiqui sara mahmood syed mohsin ali shah chamini silva laksara silva manoja gamage menka loomba vindya rajakaruna abu hanif rajan babu kamalesh balachandran kumarendran marie loh archa misra asma tassawar akansha tyagi swati waghdhare saira burney sajjad ahmad viswanathan mohan malabika sarker ian goon anuradhani kasturiratne jaspal kooner prasad katulanda sujeet jha ranjit mohan anjana malay mridha franco sassi john chambers behalf nihr global health research unit diabetes cardiovascular disease south asia pii s2352 reference ssmph appear ssm population health received date november revised date janu

In [ ]:
import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

#T-5 Summarizer

In [ ]:
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cpu')

In [ ]:
input_ids = tokenizer("summarize: "+cleaned_text, return_tensors="pt",max_length=2048, truncation=True).input_ids  # Batch size 1

outputs = model.generate(input_ids,num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=200,
                                    max_length=250,)

decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(decoded)

Keyword arguments {'Batch': 5} not recognized.


icmje criteria please check apply page low uptake covid prevention behaviours high socioeconomic impact lockdown measures south asia evidence large scale multi country surveillance programme journal pre proof. dian kusuma rajendra pradeepa, khadija, mahmood syed mohsin ali shah chamini silva laksara, manoja gamage menka loomba vindya malay mridha franco 'home'  g nypd''. high d.p' co &... high-quality i'ds y ad: o.d; c ng r t b p essm vs co-sympathetic k.i hraasiyidd


#GPT 2 Summarizer


In [ ]:
from transformers import GPT2Tokenizer,GPT2LMHeadModel
tokenizer=GPT2Tokenizer.from_pretrained('gpt2')
model=GPT2LMHeadModel.from_pretrained('gpt2')
inputs=tokenizer.batch_encode_plus([cleaned_text],return_tensors='pt',max_length=512)
summary_ids=model.generate(inputs['input_ids'],early_stopping=True)
GPT_summary=tokenizer.decode(summary_ids[0],skip_special_tokens=True)
print(GPT_summary)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 512, but ``max_length`` is set to 20.This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.


low uptake covid prevention behaviours high socioeconomic impact lockdown measures south asia evidence large scale multi country surveillance programme journal pre proof low uptake covid prevention behaviours high socioeconomic impact lockdown measures south asia evidence large scale multi country surveillance programme dian kusuma rajendra pradeepa khadija khawaja mehedi hasan samreen siddiqui sara mahmood syed mohsin ali shah chamini silva laksara silva manoja gamage menka loomba vindya rajakaruna abu hanif rajan babu kamalesh balachandran kumarendran marie loh archa misra asma tassawar akansha tyagi swati waghdhare saira burney sajjad ahmad viswanathan mohan malabika sarker ian goon anuradhani kasturiratne jaspal kooner prasad katulanda sujeet jha ranjit mohan anjana malay mridha franco sassi john chambers behalf nihr global health research unit diabetes cardiovascular disease south asia pii s2352 reference ssmph appear ssm population health received date november revised date janua

In [ ]:
GPT_summary

'low uptake covid prevention behaviours high socioeconomic impact lockdown measures south asia evidence large scale multi country surveillance programme journal pre proof low uptake covid prevention behaviours high socioeconomic impact lockdown measures south asia evidence large scale multi country surveillance programme dian kusuma rajendra pradeepa khadija khawaja mehedi hasan samreen siddiqui sara mahmood syed mohsin ali shah chamini silva laksara silva manoja gamage menka loomba vindya rajakaruna abu hanif rajan babu kamalesh balachandran kumarendran marie loh archa misra asma tassawar akansha tyagi swati waghdhare saira burney sajjad ahmad viswanathan mohan malabika sarker ian goon anuradhani kasturiratne jaspal kooner prasad katulanda sujeet jha ranjit mohan anjana malay mridha franco sassi john chambers behalf nihr global health research unit diabetes cardiovascular disease south asia pii s2352 reference ssmph appear ssm population health received date november revised date janu